### <font color="orange">ライブラリインポート</font>

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

### <font color="orange">ファイルの読み込み</font>

In [3]:
# csvファイルの読み込み
data = pd.read_csv("../../../data/Ohtani_all_season_pitch.csv")
data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2023-08-23,94.2,-1.98,5.78,"Ohtani, Shohei",687952,660271,NaN,foul,NaN,NaN,NaN,NaN,5,Christian Encarnacion-Strand grounds into a do...,R,R,R,LAA,CIN,S,NaN,NaN,2,2,2023,-0.61,1.05,-0.11,2.14,NaN,NaN,668715.0,1,2,Top,NaN,NaN,NaN,NaN,681351,NaN,NaN,6.271016,-136.825617,-6.071223,-9.087423,33.047214,-17.755643,3.36,1.55,211.0,73.8,32.0,94.6,2289.0,6.8,716890,660271,681351,592273,650859,500871,623205,545341,666160,592669,53.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,5,4-Seam Fastball,2,0,0,2,0,2,0,2,Standard,Standard,204.0,0.0,0.000
1,ST,2023-08-23,76.1,-2.01,5.74,"Ohtani, Shohei",687952,660271,NaN,swinging_strike,NaN,NaN,NaN,NaN,14,Christian Encarnacion-Strand grounds into a do...,R,R,R,LAA,CIN,S,NaN,NaN,2,1,2023,1.20,0.09,0.89,2.39,NaN,NaN,668715.0,1,2,Top,NaN,NaN,NaN,NaN,681351,NaN,NaN,3.929574,-110.687894,0.023599,8.873675,24.053258,-31.752241,3.36,1.55,NaN,NaN,NaN,75.7,1217.0,6.5,716890,660271,681351,592273,650859,500871,623205,545341,666160,592669,54.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,4,Sweeper,2,0,0,2,0,2,0,2,Standard,Standard,65.0,0.0,-0.066
2,ST,2023-08-23,77.2,-2.05,5.60,"Ohtani, Shohei",687952,660271,NaN,swinging_strike,NaN,NaN,NaN,NaN,6,Christian Encarnacion-Strand grounds into a do...,R,R,R,LAA,CIN,S,NaN,NaN,2,0,2023,0.75,0.55,0.80,2.11,NaN,NaN,668715.0,1,2,Top,NaN,NaN,NaN,NaN,681351,NaN,NaN,4.735244,-112.322517,-1.350023,5.239909,23.744398,-27.537504,3.36,1.55,NaN,NaN,NaN,77.2,2188.0,6.7,716890,660271,681351,592273,650859,500871,623205,545341,666160,592669,53.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,3,Sweeper,2,0,0,2,0,2,0,2,Standard,Standard,69.0,0.0,-0.071
3,FS,2023-08-23,90.1,-1.94,5.69,"Ohtani, Shohei",687952,660271,NaN,ball,NaN,NaN,NaN,NaN,14,Christian Encarnacion-Strand grounds into a do...,R,R,R,LAA,CIN,B,NaN,NaN,1,0,2023,-1.41,0.58,0.59,1.01,NaN,NaN,668715.0,1,2,Top,NaN,NaN,NaN,NaN,681351,NaN,NaN,9.369304,-130.719148,-6.852454,-18.235704,29.715268,-24.197503,3.39,1.53,NaN,NaN,NaN,90.5,1000.0,6.9,716890,660271,681351,592273,650859,500871,623205,545341,666160,592669,53.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,2,Split-Finger,2,0,0,2,0,2,0,2,Standard,Standard,317.0,0.0,0.058
4,FS,2023-08-23,90.0,-1.83,5.76,"Ohtani, Shohei",687952,660271,NaN,blocked_ball,NaN,NaN,NaN,NaN,14,Christian Encarnacion-Strand grounds into a do...,R,R,R,LAA,CIN,B,NaN,NaN,0,0,2023,-1.36,0.72,1.29,0.18,NaN,NaN,668715.0,1,2,Top,NaN,NaN,NaN,NaN,681351,NaN,NaN,10.723708,-130.429982,-9.447507,-17.733742,27.049300,-22.053818,3.49,1.53,NaN,NaN,NaN,90.8,1045.0,7.0,716890,660271,681351,592273,650859,500871,623205,545341,666160,592669,53.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,1,Split-Finger,2,0,0,2,0,2,0,2,Standard,Standard,191.0,0.0,0.043


### <font color="orange">使用するカラムのリストを選択</font>

In [4]:
# 必要なカラムのみを選択
select_column_list = [
#$ 試合情報
    'pitch_name',    # <- target
    'game_date',
    'home_team',
    'home_score',
    'away_score',
    
#$ 投球時の情報
    'stand',
    'on_3b',
    'on_2b',
    'on_1b',
    'inning',
    'pitch_number',
    'outs_when_up',
    'balls',
    'strikes',
    'sz_top',
    'sz_bot',
    
#$ 一つ前の投球の情報として扱うカラム    
    'release_speed',
    'release_pos_x',
    'release_pos_y',
    'release_pos_z',
    'release_spin_rate',
    'release_extension',
    'plate_x',
    'plate_z',
    'events',
    'description',
    'zone',
    'type',
    'hc_x',
    'hc_y'
]

### <font color="orange">データ整形</font>

In [5]:
data = pd.read_csv("../../../data/Ohtani_all_season_pitch.csv")
# 使用するカラム抽出
data = data[select_column_list]

def make_data(data):
    """データフレームを受け取り、所望の形に整形する"""
    
    df = data.copy()

    # データを時系列順にソート
    df = df.sort_index(ascending=False)

    # game_dateをyear, month, dateに分割
    df["game_date"] = pd.to_datetime(df["game_date"])
    df.insert(2, "game_year", df["game_date"].dt.strftime("%Y").astype(int))
    df.insert(3, "game_month", df["game_date"].dt.strftime("%m").astype(int))
    df.insert(4, "game_day", df["game_date"].dt.strftime("%d").astype(int))
    del df["game_date"]
    
    # home_teamのエンコーディング（エンゼルスがホームなら0, アウェイなら1に変換
    df[df[["home_team"]] != "LAA"] = 1 
    df[df[["home_team"]] == "LAA"] = 0 
    
    # home_score, away_scoreをエンゼルスの得点と、敵チームの得点に変更する。
    df.insert(7, "LAA_score", 0)
    df.insert(8, "ENEMY_score", 0)
    
    df.loc[df["home_team"] == 0, "LAA_score"] = df["home_score"]
    df.loc[df["home_team"] == 0, "ENEMY_score"] = df["away_score"]
    df.loc[df["home_team"] == 1, "LAA_score"] = df["away_score"]
    df.loc[df["home_team"] == 1, "ENEMY_score"] = df["home_score"]
    del df["home_score"]
    del df["away_score"]
    
    # stand のエンコーディング
    # R（右打者）-> 0, L（左打者）-> 1にエンコーディング
    stand_encording = {"R" : 0, "L" : 1}
    df["stand"] = df["stand"].map(stand_encording)
    
    # on_3b, on_2b, on_1bのエンコーディング
    # NaN -> 0, not NaN -> 1
    df["on_1b"] = df["on_1b"].map(lambda x : 0 if pd.isna(x) else 1)
    df["on_2b"] = df["on_2b"].map(lambda x : 0 if pd.isna(x) else 1) 
    df["on_3b"] = df["on_3b"].map(lambda x : 0 if pd.isna(x) else 1)
    
    # events, descriptionをラベルエンコーディング
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df["events"] = le.fit_transform(df["events"])
    df["description"] = le.fit_transform(df["description"])
    
    # typeをエンコーディング
    type_encording = {"S" : 0, "B" : 1, "X" : 2}
    df["type"] = df["type"].map(type_encording)
    
    # hc_x, hc_yの欠損値を0で補完
    df["hc_x"] = df["hc_x"].fillna(0)
    df["hc_y"] = df["hc_y"].fillna(0)
    
    return df

data = make_data(data)

In [6]:
# 時系列に矛盾がないか確認
data.select_dtypes(include=int).diff().query("game_day == 0 and (LAA_score < 0 or ENEMY_score < 0)")

,game_year,game_month,game_day,LAA_score,ENEMY_score,stand,on_3b,on_2b,on_1b,inning,pitch_number,outs_when_up,balls,strikes,events,description,zone,type


### <font color="orange">yearごとの投球数を確認（投球数が少ない年は削除）</font>

In [7]:
# 年ごとの投球数を確認
data.game_year.value_counts(sort=False)

game_year
2018     853
2020      80
2021    2027
2022    2629
2023    2094
Name: count, dtype: int64

In [8]:
data = data[data["game_year"].isin([2021, 2022, 2023])]

# 確認
data.game_year.value_counts(sort=False)

game_year
2021    2027
2022    2629
2023    2094
Name: count, dtype: int64

### <font color="orange">欠損値の確認</font>

In [9]:
na_counts = data.isna().sum()
na_counts[na_counts > 0]

release_spin_rate    31
release_extension     1
dtype: int64

In [10]:
# release_spin_rate（ボールの回転数）とrelease_extension（投手のホーム方向へのリリース位置）の欠損値補完をする。
# 欠損値を球種ごとの平均値で補完。

def missing_value_imputation_release_spin_rate(row):
    if np.isnan(row["release_spin_rate"]):
        tmp_df = data[data["pitch_name"]==row["pitch_name"]]
        mean_release_spin_rate = tmp_df["release_spin_rate"].mean()
        return mean_release_spin_rate.round(1)
    else:
        return row["release_spin_rate"]
    
    
def missing_value_imputation_release_extension(row):
    if np.isnan(row["release_extension"]):
        tmp_df = data[data["pitch_name"]==row["pitch_name"]]
        mean_release_spin_rate = tmp_df["release_extension"].mean()
        return mean_release_spin_rate.round(1)
    else:
        return row["release_extension"]

data["release_spin_rate"] = data.apply(missing_value_imputation_release_spin_rate, axis=1)
data["release_extension"] = data.apply(missing_value_imputation_release_extension, axis=1)

In [11]:
data.isna().sum().sum()

0

### <font color="orange">球種ごとの投球数確認</font>

In [12]:
data.groupby("game_year")["pitch_name"].value_counts(normalize=True).round(2).to_frame()

proportion
game_year pitch_name                 
2021      4-Seam Fastball        0.44
          Sweeper                0.22
          Split-Finger           0.18
          Cutter                 0.12
          Curveball              0.03
          Slider                 0.00
2022      Sweeper                0.37
          4-Seam Fastball        0.27
          Split-Finger           0.12
          Cutter                 0.09
          Curveball              0.08
          Sinker                 0.04
          Slider                 0.02
          Slow Curve             0.00
2023      Sweeper                0.35
          4-Seam Fastball        0.33
          Cutter                 0.16
          Split-Finger           0.06
          Sinker                 0.06
          Curveball              0.04
          Slider                 0.00

### <font color="orange">時系列データの整形</font>

In [13]:
#$ 一つ前の投球の情報として扱うカラム
ls_time_series = [
    'release_speed',
    'release_pos_x',
    'release_pos_y',
    'release_pos_z',
    'release_spin_rate',
    'release_extension',
    'plate_x',
    'plate_z',
    'events',
    'description',
    'zone',
    'type',
    'hc_x',
    'hc_y'
]

# 下方向に1行ずらし、一球前の情報として扱う。
data[ls_time_series] = data[ls_time_series].shift()
# アウトやヒットなどで打者が変わった時（pitch_number==1）のとき、一球前の情報はないものとして、
# is_first_pitch_numberでフラグを立て、欠損値には外れ値を代入する。
data.loc[data["pitch_number"]==1, ls_time_series] = pd.NA
data.insert(18, "is_first_pitch_number", 0)
data.loc[data["pitch_number"]==1, "is_first_pitch_number"] = 1

data.head(50)

,pitch_name,game_year,game_month,game_day,home_team,LAA_score,ENEMY_score,stand,on_3b,on_2b,on_1b,inning,pitch_number,outs_when_up,balls,strikes,sz_top,sz_bot,is_first_pitch_number,release_speed,release_pos_x,release_pos_y,release_pos_z,release_spin_rate,release_extension,plate_x,plate_z,events,description,zone,type,hc_x,hc_y
6749,4-Seam Fastball,2021,4,4,0,0,0,0,0,0,0,1,1,0,0,0,3.45,1.61,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6748,4-Seam Fastball,2021,4,4,0,0,0,0,0,0,0,1,2,0,1,0,3.42,1.56,0,98.2,-1.92,53.47,5.96,2457.0,7.0,0.25,1.42,19.0,0.0,14.0,1.0,0.0,0.0
6747,4-Seam Fastball,2021,4,4,0,0,0,0,0,0,0,1,3,0,2,0,3.41,1.56,0,96.9,-1.94,53.78,6.00,2358.0,6.7,-2.06,3.15,19.0,0.0,11.0,1.0,0.0,0.0
6746,4-Seam Fastball,2021,4,4,0,0,0,0,0,0,0,1,4,0,2,1,3.41,1.56,0,96.4,-1.90,53.64,6.03,2411.0,6.9,-0.21,3.39,19.0,9.0,2.0,0.0,0.0,0.0
6745,Sweeper,2021,4,4,0,0,0,0,0,0,0,1,5,0,2,2,3.41,1.56,0,98.9,-1.88,53.66,6.00,2377.0,6.8,-0.80,1.44,19.0,3.0,13.0,0.0,0.0,0.0
6744,Curveball,2021,4,4,0,0,0,1,0,0,0,1,1,1,0,0,3.18,1.50,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6743,4-Seam Fastball,2021,4,4,0,0,0,1,0,0,0,1,2,1,1,0,3.29,1.55,0,75.5,-1.90,53.90,6.20,2545.0,6.6,-3.42,4.78,19.0,0.0,11.0,1.0,0.0,0.0
6742,4-Seam Fastball,2021,4,4,0,0,0,1,0,0,0,1,3,1,2,0,3.12,1.55,0,97.9,-1.89,53.50,6.00,2390.0,7.0,1.46,0.90,19.0,0.0,14.0,1.0,0.0,0.0
6741,4-Seam Fastball,2021,4,4,0,0,0,1,0,0,0,1,4,1,2,1,3.29,1.49,0,97.2,-2.10,53.79,5.98,2358.0,6.7,-0.52,2.72,19.0,2.0,1.0,0.0,0.0,0.0
6740,4-Seam Fastball,2021,4,4,0,0,0,1,0,0,0,1,5,1,2,2,3.29,1.49,0,99.1,-2.04,53.65,5.98,2496.0,6.9,0.10,3.32,19.0,9.0,2.0,0.0,0.0,0.0


### <font color="orange">正解ラベルのエンコーディング</font>

In [14]:
data.groupby("game_year")["pitch_name"].value_counts(normalize=True).round(2).to_frame()

proportion
game_year pitch_name                 
2021      4-Seam Fastball        0.44
          Sweeper                0.22
          Split-Finger           0.18
          Cutter                 0.12
          Curveball              0.03
          Slider                 0.00
2022      Sweeper                0.37
          4-Seam Fastball        0.27
          Split-Finger           0.12
          Cutter                 0.09
          Curveball              0.08
          Sinker                 0.04
          Slider                 0.02
          Slow Curve             0.00
2023      Sweeper                0.35
          4-Seam Fastball        0.33
          Cutter                 0.16
          Split-Finger           0.06
          Sinker                 0.06
          Curveball              0.04
          Slider                 0.00

In [15]:
# pitch_nameをエンコーディング
pitch_name_encording = {
    "Sweeper" : 0,
    "4-Seam Fastball" : 1,
    "Split-Finger" : 2,
    "Cutter" : 2,
    "Curveball" : 2,
    "Sinker" : 2,
    "Slider" : 2,
    "Slow Curve" : 2
}

data.loc[data["game_year"]==2021, "pitch_name"] = data["pitch_name"].map(pitch_name_encording)
data.loc[data["game_year"]==2022, "pitch_name"] = data["pitch_name"].map(pitch_name_encording)
data.loc[data["game_year"]==2023, "pitch_name"] = data["pitch_name"].map(pitch_name_encording)

data["pitch_name"] = data["pitch_name"].astype(int)

data["pitch_name"].value_counts()

pitch_name
1    2296
2    2293
0    2161
Name: count, dtype: int64

In [16]:
data.groupby("game_year")["pitch_name"].value_counts(normalize=True).round(2).to_frame()

proportion
game_year pitch_name            
2021      1                 0.44
          2                 0.34
          0                 0.22
2022      0                 0.37
          2                 0.35
          1                 0.27
2023      0                 0.35
          1                 0.33
          2                 0.32

In [17]:
# 2021年は2022年や2023年と比較して配球比率が大きく異なるので削除する。
data = data[data["game_year"].isin([2022, 2023])]

In [18]:
# 確認
data["game_year"].unique()

array([2022, 2023])

In [19]:
data.to_csv("./maked_data.csv", index=False)